In [236]:
import pandas as pd
from Bio import SeqIO
import numpy as np
import networkx as nx
import scipy.sparse as sp
from Bio import SeqIO

np.random.seed(seed=2)

In [257]:
def split_train_test(fasta, train_frac=1, test_frac=0.1 out=False):
    df_edges = pd.read_csv('../Data/Interactome/huri_apid_merge_ppis_edgelist.csv', sep='\t', header=None)
    df_nodes = pd.read_csv('../Data/Interactome/huri_apid_merge_ppis_nodelist.csv', sep='\t', header=None)
#     df_mmseq = pd.read_csv('../Data/mmseqs/align.m8', sep='\t', header=None)
#     df_clu = pd.read_csv('../Data/mmseqs/DB_clu.tsv', sep='\t', header=None)
    red_prot = [x.id.split('|')[1] for x in fasta]
    
    edges = np.array([x for x in df_edges.values if x[0] in red_prot and x[1] in red_prot])
    np.random.shuffle(edges)
    
    num_test = int(len(edges) * test_frac)
    train_edges = edges[num_test:]
    np.random.shuffle(train_edges)
    train_edges = train_edges[:int(len(train_edges)*train_frac)]
    test_edges = edges[:num_test]
    
    train_nodes = set(train_edges.flatten())
    test_nodes = set(test_edges.flatten())
    
    c1 = [x for x in test_edges if x[0] in train_nodes and x[1] in train_nodes]
    c2 = [x for x in test_edges if x[0] not in train_nodes or x[1] not in train_nodes]
    c3 = [x for x in test_edges if x[0] not in train_nodes and x[1] not in train_nodes]
    
    if out:
        np.savez('../Data/train_test_data/train_edges.npz', train_edges)
        np.savez('../Data/train_test_data/test_edges.npz', test_edges)
    
    print('Train:',len(train_edges),'C1:',len(c1),'C2:', len(c2),'C3:', len(c3))

In [258]:
fasta = list(SeqIO.parse("../Data/Fasta/huri_apid_proteins.fasta", "fasta"))
fasta = list(SeqIO.parse("../Data/test/huri_apid40.fasta", "fasta"))

split_train_test(fasta, 0.2)

Train: 9160 C1: 3814 C2: 1274 C3: 108


In [404]:
fasta = list(SeqIO.parse("../Data/Fasta/huri_apid_proteins.fasta", "fasta"))
df_edges = pd.read_csv('../Data/Interactome/huri_apid_merge_ppis_edgelist.csv', sep='\t', header=None)
df_nodes = pd.read_csv('../Data/Interactome/huri_apid_merge_ppis_nodelist.csv', sep='\t', header=None)
df_mmseq = pd.read_csv('../Data/mmseqs/align.m8', sep='\t', header=None)

red_prot = [x.id.split('|')[1] for x in fasta]
edges = np.array([x for x in df_edges.values if x[0] in red_prot and x[1] in red_prot])


In [405]:
np.random.shuffle(edges)

num_test = int(len(edges) * 0.25)
train_edges = edges[num_test:]
np.random.shuffle(train_edges)
train_edges = train_edges[:int(len(train_edges)*1)]
test_edges = edges[:num_test]
    
G = nx.read_edgelist('../Data/Interactome/huri_apid_merge_ppis_edgelist.csv')

nr_train_edges = []
for edge in train_edges:
    if not G.has_edge(edge[0], edge[1]):
        continue
    nr_train_edges.append(edge)   
    G.remove_nodes_from(edge)

train_nodes = set(np.array(nr_train_edges).flatten())
test_nodes = set(test_edges.flatten())

In [406]:
more_40_seq_id = df_mmseq[df_mmseq[2] > 0.4]
node_sim_dict = {}
for x in more_40_seq_id[[0,1]].values:
    if x[0] not in node_sim_dict.keys():
        node_sim_dict[x[0]] = [x[0]]
    if x[1] not in node_sim_dict.keys():
        node_sim_dict[x[1]] = [x[1]]
    
    node_sim_dict[x[0]].append(x[1])
    node_sim_dict[x[1]].append(x[0])

node_sim_dict = {x:list(set(y)) for x,y in node_sim_dict.items()}

In [407]:
sim_train_nodes = []
for x in train_nodes:
    if x in node_sim_dict.keys():
        sim_train_nodes = sim_train_nodes + node_sim_dict[x]
    else:
        sim_train_nodes = sim_train_nodes + [x]

In [408]:
c1 = [x for x in test_edges if x[0] in sim_train_nodes and x[1] in sim_train_nodes]
c2 = [x for x in test_edges if x[0] not in sim_train_nodes or x[1] not in sim_train_nodes]
c3 = [x for x in test_edges if x[0] not in sim_train_nodes and x[1] not in sim_train_nodes]

In [409]:
len(keep_edges)

4408

In [410]:
len(test_edges)

27934

In [411]:
len(c1)

24091

In [412]:
len(c2)

3843

In [413]:
len(c3)

126